In [ ]:
"""Full dataframe logger from xml file"""



def pull_marker_data(xml_file):
    import xml.etree.ElementTree as et
    import pandas as pd
    file = et.parse(xml_file)
    root = file.getroot()
    marker_data = []

    MDElement = root.find('Marker_Data')
    for Marker_Type in MDElement.findall('Marker_Type'):
        type_id = int(Marker_Type.find('Type').text)

        for Marker in Marker_Type.findall('Marker'):
            xloc = int(Marker.find('MarkerX').text) if Marker.find('MarkerX') is not None else None
            yloc = int(Marker.find('MarkerY').text) if Marker.find('MarkerY') is not None else None
            zloc = int(Marker.find('MarkerZ').text) if Marker.find('MarkerZ') is not None else None
            zlayer = (2 + zloc) // 3

            marker_data.append({
                'z': zlayer,
                'type_id': type_id,
                'y': yloc,
                'x': xloc,
            })

    dataframe = pd.DataFrame(marker_data, columns=['z', 'type_id', 'y', 'x'])
    return dataframe

result = pull_marker_data('FF15A/CellCounter_FF151L2L.xml')
result.head(96)

In [ ]:
""" create the most distinct colors from the dataframe"""



def find_distinct_colors(xml_file):
    import numpy as np
    import colorsys

    df = pull_marker_data(xml_file)
    num_colors = max(df['type_id'])

    colors = []
    for i in range(num_colors):
        r, g, b = colorsys.hsv_to_rgb(i / num_colors, 1.0, 1.0)
        rgb = np.array([int(r * 255), int(g * 255), int(b * 255)])
        colors.append(rgb)

    return colors


print(find_distinct_colors('FF15A/CellCounter_FF151L2L.xml'))


In [ ]:
"""full 3 channel hue color tiff writer from dataframe log"""

"full working code readable on napari image viewer"

def pull_synapse_and_mark_file(xml_file, input_file, output_file):
    import numpy as np
    import tifffile
    df = pull_marker_data(xml_file)
    color_array = find_distinct_colors(xml_file)
    original_image = tifffile.imread(input_file)
    layers, channels, height, width = original_image.shape
    rgb_image = np.zeros((layers, 3, height, width), dtype=np.uint8)
    x = df['x']
    y = df['y']
    z = df['z']
    type_id = df['type_id']
    for i in range(len(df)):
        rgb_image[z[i], :, y[i], x[i]] = color_array[type_id[i]-1]
    tifffile.imwrite(output_file, rgb_image)
    return rgb_image

pull_synapse_and_mark_file('FF15A/CellCounter_FF151L2L.xml', 'FF15/FF151L2L.lsm', 'FF15/FF151L2L_marked94.tif')


In [ ]:
"""automated image marker from folder"""

def mark_folder(file_folder):
    import os
    import numpy as np
    import tifffile

    folder = file_folder
    output = folder + "_markers"
    os.makedirs(output, exist_ok=True)
    lsm_list = [file for file in os.listdir(folder) if file.endswith(".lsm") and not (file.startswith(".") or file.endswith("old.lsm"))]
    if len(lsm_list) == 0:
        print("No .lsm files found in the folder.")
        return
    else:
        for file in lsm_list:
            print(file)
            image = tifffile.imread(folder + "/" + file)
            layers, channels, height, width = image.shape
            rgb_image = np.zeros((layers, channels, height, width), dtype=np.uint8)
            xml_file_path = folder + "A/CellCounter_" + file.replace(".lsm", ".xml")
            xml_alternate_path = folder + "A_/CellCounter_" + file.replace(".lsm", ".xml")
            xml_window_path = folder + "A/CellCounter_Counter Window - " + file.replace(".lsm", ".xml")
            if os.path.exists(xml_file_path):
                df = pull_marker_data(xml_file_path)
                color_array = find_distinct_colors(xml_file_path)
            else:
                if os.path.exists(xml_alternate_path):
                    df = pull_marker_data(xml_alternate_path)
                    color_array = find_distinct_colors(xml_alternate_path)
                else:
                    if os.path.exists(xml_window_path):
                        df = pull_marker_data(xml_window_path)
                        color_array = find_distinct_colors(xml_window_path)
                    else:
                        print(xml_file_path + "does not exist. Skipping file.")

            x = df['x']
            y = df['y']
            z = df['z']
            type_id = df['type_id']
            filename = file.replace(".lsm", "_marked.tif")
            filepath = os.path.join(output, filename)
            for i in range(len(df)):
                rgb_image[z[i]-1, :, y[i]-1, x[i]-1] = color_array[type_id[i]-1]
            tifffile.imwrite(filepath, rgb_image)

In [ ]:
import os
path = r"C:\Users\richa\parralabimageprocessing\Synapse Confocal Images Rat"
for folder in os.listdir(path):
    if not (folder.startswith(".") or folder.startswith("w")):
        mark_folder(os.path.join(path, folder))
